In [1]:
import openai
openai.api_key = open("key.txt","r").read().strip('\n')

In [2]:
completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages = [{"role": "user", "content": "How can i use Chatgpt-api"}]
)

print(completion)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "You can use the ChatGPT API by deploying it to create conversational agents, draft emails, write code, and more. Here's how to use it:\n\n1. First you need to have an API key. You get this key by subscribing to the API on OpenAI.\n\n2. Install \u201copenai\u201d python package with pip:\n\n       pip install openai\n\n3. Import OpenAI's GPT-3 into your python code:\n\n       import openai\n\n4. Make sure to set the API key:\n\n      openai.api_key = 'your-api-key'\n\n5. You can now use the `'openai.ChatCompletion.create'` method to create a chat message. It takes in a model name (like \"text-davinci-002\") and a list of messages.\n\n      Example in Python:\n\n          response = openai.ChatCompletion.create(\n          model=\"gpt-3.5-turbo\",\n          messages=[\n              {\"role\": \"system\", \"content\": \"You are a helpful assistant.\"},\n

In [3]:
reply = completion['choices'][0].message.content
print(reply)

You can use the ChatGPT API by deploying it to create conversational agents, draft emails, write code, and more. Here's how to use it:

1. First you need to have an API key. You get this key by subscribing to the API on OpenAI.

2. Install “openai” python package with pip:

       pip install openai

3. Import OpenAI's GPT-3 into your python code:

       import openai

4. Make sure to set the API key:

      openai.api_key = 'your-api-key'

5. You can now use the `'openai.ChatCompletion.create'` method to create a chat message. It takes in a model name (like "text-davinci-002") and a list of messages.

      Example in Python:

          response = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=[
              {"role": "system", "content": "You are a helpful assistant."},
              {"role": "user", "content": "Who won the world series in 2020?"},
          ])

You’ll see the assistant’s reply in the `response['choices'][0]['message']['content’]` 

In [4]:
!pip install python-dotenv
!pip install matplotlib
!pip install pydantic
!pip install pyecharts
!pip install fastapi

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [5]:
#PLOT GPT generated top station data
import re
import matplotlib.pyplot as plt
from pyecharts.charts import Bar, Page, Pie, Grid
from pyecharts import options as opts



# Define the function to plot the gpt data
def plot_gpt_data_to_html(filtered_gpt_data):
    data = filtered_gpt_data

    # Updated pattern to handle both data formats
    pattern = r'(\w[\w\s,]*[^\s])\s\((\d+)\)|\(([\w\s,]+), (\d+)\)'
    matches = re.findall(pattern, data)

    # Extracting station names and frequencies considering both matching groups from the updated pattern
    stations = [match[0] if match[2] == '' else match[2] for match in matches]
    frequencies = [int(match[1]) if match[3] == '' else int(match[3]) for match in matches]

    bar_chart = (
        Bar()
        .add_xaxis(stations)
        .add_yaxis("Frequenz", frequencies, tooltip_opts=opts.TooltipOpts(is_show=True, trigger="axis", axis_pointer_type="cross"))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Top 10 Betroffenen Stationen"),  
            xaxis_opts=opts.AxisOpts(
                name="Stationen",  # X-Axis title
                axislabel_opts=opts.LabelOpts(
                    rotate=45,   # Try 45-degree rotation
                    font_size=8  # Further reduce font size
                )
            ),
            yaxis_opts=opts.AxisOpts(
                name="Frequenz",  # Y-Axis title
            )
        )
    )

    return bar_chart.render_notebook()



# Define the function to plot the line data
def plot_line_data_to_html(lines_data_list):
    data = lines_data_list
    page = Page(layout=Page.SimplePageLayout)
    bars = []

    for idx, week_data in enumerate(data, 1):
        labels, values = zip(*week_data)
        bar_chart = (
            Bar()
            .add_xaxis(list(labels))
            .add_yaxis(f"Woche {idx}", list(values), label_opts=opts.LabelOpts(is_show=False))
            .set_global_opts(
                title_opts=opts.TitleOpts(title=f"Woche {idx} Vorfälle je Linie"),
                xaxis_opts=opts.AxisOpts(
                    name="Linie",  # X-Axis title
                    axislabel_opts=opts.LabelOpts(rotate=45)  # Optional: Rotate x-axis labels if needed
                ),
                yaxis_opts=opts.AxisOpts(
                    name="Anzahl Vorfälle"  # Y-Axis title
                )
            )
        )
        bars.append(bar_chart)

    page.add(*bars)
    
    print("Generating plot...")
    # Return the embeddable HTML
    #return page.render_embed()
    return page.render_notebook()

# Define the function to plot the accident data
def plot_accident_data_to_html(accident_data_list):
    data = accident_data_list

    def generate_pie(data_item, title):
        categories = list(data_item.keys())
        values = list(data_item.values())
        pie = (
            Pie()
            .add("", [list(z) for z in zip(categories, values)])
            .set_global_opts(title_opts=opts.TitleOpts(title=title), legend_opts=opts.LegendOpts(orient="vertical", pos_top="15%", pos_left="85%"))
            .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {d}%"))  # Displaying category name and percentage
        )
        return pie

    page = Page()
    for index, data_item in enumerate(data):
        page.add(generate_pie(data_item, f"Woche {index+1}"))
    
    

        
    # Return the embeddable HTML
    #return page.render_embed()
    return page.render_notebook()


In [6]:
import json
from dotenv import load_dotenv
from datetime import datetime, timedelta
import openai
import os
from pyecharts import *
#from plot import plot_line_data_to_html, plot_accident_data_to_html, plot_gpt_data_to_html 

load_dotenv()
# Configure the OpenAI API client
#openai.api_type = os.getenv("OPENAI_API_TYPE")
#openai.api_version = os.getenv("OPENAI_API_VERSION")
#openai.api_base = os.getenv("OPENAI_API_BASE")  # Your Azure OpenAI resource's endpoint value.
#openai.api_key = os.getenv("OPENAI_API_KEY")

# Define the function to generate the response
#def generate_response(resulted_data):
#    response = openai.ChatCompletion.create(
#        model="gpt-4",
        #engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
#        temperature = 0.4,
#        messages=[
#            {"role": "system", "content": "Du bist ein Analyse-Chatbot. Aus den bereitgestellten JSON-Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-JSON-Daten:"+ resulted_data},
#            {"role": "user", "content": "Im JSON-Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
#            }
#        ]
#    )
#    return response['choices'][0]['message']['content']

def generate_response(resulted_data):
    response = openai.ChatCompletion.create(
    model="gpt-4-0314",
    # engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
    temperature=0,
    messages=[
      {"role": "user",
       "content": "Du bist ein Analyst. Aus den bereitgestellten Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-List-Daten:" + resulted_data + "Im Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
      }
    ]
  )
    return response['choices'][0]['message']['content']


# Define the function to filter the accident data
def get_incident_data(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    #gpt filter
    filtered_incidents = [
        {
            'title': entry['title'],
            'description': entry['description']
        } 
        for entry in data['incidents'] 
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime
    ]
        
    resulted_data = json.dumps(filtered_incidents)
    
    print(resulted_data)
    
    #filtered_gpt_data = ''
    filtered_gpt_data = generate_response(resulted_data)
    
    return filtered_gpt_data



# Define the function to filter the line data 
def get_top10_lines(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    filtered_lines_incidents = []
    for entry in data['incidents']:
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime:
            filtered_lines_incidents.append(entry['lines'])

    from collections import Counter


    # Flatten the list of lists
    flattened_list = [item for sublist in filtered_lines_incidents for item in sublist]
    # Calculate the frequency of each element
    element_frequencies = Counter(flattened_list)
    # Get the top 10 most common elements and their frequencies
    top_10 = element_frequencies.most_common(10)

    return top_10

# Define the function to filter the accident time data
def get_accident_time_sum(data, start_time, end_time):
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)

    filtered_accident_sum = {"Verkehrsunfall": 0, "Falschparker": 0}
    #for entry in data['incidents']:
    for entry in data['incidents']:
        # Data validation for the 'start' and 'end' timestamps
        if 'start' not in entry or 'end' not in entry:
            #print("Missing 'start' or 'end' in entry:", entry)
            continue
        if not entry['start'] or not entry['end']:
            #print("Empty 'start' or 'end' timestamp in entry:", entry)
            continue

        try:
            start_incident_datetime = datetime.strptime(entry['start'], date_format)
            end_incident_datetime = datetime.strptime(entry['end'], date_format)
        except ValueError as e:
            #print(f"Error parsing timestamp in entry {entry}: {e}")
            continue

        # Check the incident title and compute the time sum
        if start_datetime <= start_incident_datetime <= end_datetime and ("Verkehrsunfall" in entry['title'] or "Falschparker" in entry['title']):
            if "Verkehrsunfall" in entry['title']:
                filtered_accident_sum['Verkehrsunfall'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60
            if "Falschparker" in entry['title']:
                filtered_accident_sum['Falschparker'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60

    filtered_accident_sum = {key: (round(value, 2) if isinstance(value, float) else value) for key, value in filtered_accident_sum.items()}

    return filtered_accident_sum


# Define the function to generate the date ranges
def get_date_ranges():
    # Initialize the end date to today's date at 00:00:00
    end_time = datetime.strptime('2018-08-16 00:00:00.00000','%Y-%m-%d %H:%M:%S.%f')
    # Define the time interval (1 week = 7 days)
    interval = timedelta(days=1)
    date_ranges = []
    # Generate the ranges
    for _ in range(3):
        start_time = end_time - interval
        date_ranges.append((start_time, end_time))
        end_time = start_time
    return date_ranges


# Define the function to generate the Subway_line data
def get_line_data(date_ranges):
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)

    #top 10 line data
    lines_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filetered_top_10_line_data = get_top10_lines(data, start_str, end_str)
        #print("Line data from data analysis:")
        lines_data_list.append(filetered_top_10_line_data)
    print(lines_data_list)
    return plot_line_data_to_html(lines_data_list)


# Define the function to generate the accident data
def get_accident_data(date_ranges):
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)

    #accident time sum daata
    accident_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filtered_accident_time_data = get_accident_time_sum(data, start_str, end_str)
        #print("Filtered Accident time data")
        accident_data_list.append(filtered_accident_time_data)
    print(accident_data_list)
    return plot_accident_data_to_html(accident_data_list)


# Define the function to generate the gpt data
def get_gpt_data(date_ranges):
    # Use the list of tuples in a for loop
    data = read_data()
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        
        print(f"Start: {start_str}, End: {end_str}")
        filtered_gpt_data = get_incident_data(data, start_str, end_str)
        
        print("Filtered GPT station data:")
        print(filtered_gpt_data)
        break
    return plot_gpt_data_to_html(filtered_gpt_data)

# Define the function to read the data
def read_data():
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)
    return data 
#get_line_data(get_date_ranges())
#get_accident_data(get_date_ranges())
#get_gpt_data(get_date_ranges())


In [7]:
get_line_data(get_date_ranges())
#get_accident_data(get_date_ranges())
#get_gpt_data(get_date_ranges())

[[('9', 2), ('U1', 2), ('U3', 1), ('29A', 1), ('13A', 1), ('D', 1), ('49', 1), ('42', 1), ('18', 1), ('71', 1)], [('1', 3), ('U4', 2), ('18', 2), ('2A', 1), ('39A', 1), ('6', 1), ('62', 1), ('WLB', 1), ('2', 1), ('D', 1)], [('60', 3), ('42', 2), ('25', 2), ('26', 2), ('52', 2), ('U3', 1), ('49', 1), ('43', 1), ('2', 1), ('64A', 1)]]
Generating plot...


In [8]:
get_accident_data(get_date_ranges())

[{'Verkehrsunfall': 119.0, 'Falschparker': 372.0}, {'Verkehrsunfall': 105.0, 'Falschparker': 129.0}, {'Verkehrsunfall': 299.0, 'Falschparker': 123.0}]


In [9]:
import json
from dotenv import load_dotenv
from datetime import datetime, timedelta
import openai
import os
from pyecharts import *
#from plot import plot_line_data_to_html, plot_accident_data_to_html, plot_gpt_data_to_html 

load_dotenv()
# Configure the OpenAI API client
#openai.api_type = os.getenv("OPENAI_API_TYPE")
#openai.api_version = os.getenv("OPENAI_API_VERSION")
#openai.api_base = os.getenv("OPENAI_API_BASE")  # Your Azure OpenAI resource's endpoint value.
#openai.api_key = os.getenv("OPENAI_API_KEY")

# Define the function to generate the response
#def generate_response(resulted_data):
#    response = openai.ChatCompletion.create(
#        model="gpt-4",
        #engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
#        temperature = 0.4,
#        messages=[
#            {"role": "system", "content": "Du bist ein Analyse-Chatbot. Aus den bereitgestellten JSON-Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-JSON-Daten:"+ resulted_data},
#            {"role": "user", "content": "Im JSON-Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
#            }
#        ]
#    )
#    return response['choices'][0]['message']['content']

def generate_response(resulted_data):
    response = openai.ChatCompletion.create(
    model="gpt-4-0314",
    # engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
    temperature=0.7,
    messages=[
      {"role": "user",
       "content": "Du bist ein Analyst. Aus den bereitgestellten Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-List-Daten:" + resulted_data + "Im Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
      }
    ]
  )
    return response['choices'][0]['message']['content']


# Define the function to filter the accident data
def get_incident_data(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    #gpt filter
    filtered_incidents = [
        {
            'title': entry['title'],
            'description': entry['description']
        } 
        for entry in data['incidents'] 
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime
    ]
        
    resulted_data = json.dumps(filtered_incidents)
    
    print(resulted_data)
    
    #filtered_gpt_data = ''
    filtered_gpt_data = generate_response(resulted_data)
    
    return filtered_gpt_data



# Define the function to filter the line data 
def get_top10_lines(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    filtered_lines_incidents = []
    for entry in data['incidents']:
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime:
            filtered_lines_incidents.append(entry['lines'])

    from collections import Counter


    # Flatten the list of lists
    flattened_list = [item for sublist in filtered_lines_incidents for item in sublist]
    # Calculate the frequency of each element
    element_frequencies = Counter(flattened_list)
    # Get the top 10 most common elements and their frequencies
    top_10 = element_frequencies.most_common(10)

    return top_10

# Define the function to filter the accident time data
def get_accident_time_sum(data, start_time, end_time):
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)

    filtered_accident_sum = {"Verkehrsunfall": 0, "Falschparker": 0}
    #for entry in data['incidents']:
    for entry in data['incidents']:
        # Data validation for the 'start' and 'end' timestamps
        if 'start' not in entry or 'end' not in entry:
            #print("Missing 'start' or 'end' in entry:", entry)
            continue
        if not entry['start'] or not entry['end']:
            #print("Empty 'start' or 'end' timestamp in entry:", entry)
            continue

        try:
            start_incident_datetime = datetime.strptime(entry['start'], date_format)
            end_incident_datetime = datetime.strptime(entry['end'], date_format)
        except ValueError as e:
            #print(f"Error parsing timestamp in entry {entry}: {e}")
            continue

        # Check the incident title and compute the time sum
        if start_datetime <= start_incident_datetime <= end_datetime and ("Verkehrsunfall" in entry['title'] or "Falschparker" in entry['title']):
            if "Verkehrsunfall" in entry['title']:
                filtered_accident_sum['Verkehrsunfall'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60
            if "Falschparker" in entry['title']:
                filtered_accident_sum['Falschparker'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60

    filtered_accident_sum = {key: (round(value, 2) if isinstance(value, float) else value) for key, value in filtered_accident_sum.items()}

    return filtered_accident_sum


# Define the function to generate the date ranges
def get_date_ranges():
    # Initialize the end date to today's date at 00:00:00
    end_time = datetime.strptime('2018-08-16 00:00:00.00000','%Y-%m-%d %H:%M:%S.%f')
    # Define the time interval (1 week = 7 days)
    interval = timedelta(days=1)
    date_ranges = []
    # Generate the ranges
    for _ in range(3):
        start_time = end_time - interval
        date_ranges.append((start_time, end_time))
        end_time = start_time
    return date_ranges


# Define the function to generate the Subway_line data
def get_line_data(date_ranges):
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)

    #top 10 line data
    lines_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filetered_top_10_line_data = get_top10_lines(data, start_str, end_str)
        #print("Line data from data analysis:")
        lines_data_list.append(filetered_top_10_line_data)
    print(lines_data_list)
    return plot_line_data_to_html(lines_data_list)


# Define the function to generate the accident data
def get_accident_data(date_ranges):
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)

    #accident time sum daata
    accident_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filtered_accident_time_data = get_accident_time_sum(data, start_str, end_str)
        #print("Filtered Accident time data")
        accident_data_list.append(filtered_accident_time_data)
    print(accident_data_list)
    return plot_accident_data_to_html(accident_data_list)


# Define the function to generate the gpt data
def get_gpt_data(date_ranges):
    # Use the list of tuples in a for loop
    data = read_data()
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        
        print(f"Start: {start_str}, End: {end_str}")
        filtered_gpt_data = get_incident_data(data, start_str, end_str)
        
        print("Filtered GPT station data:")
        print(filtered_gpt_data)
        break
    return plot_gpt_data_to_html(filtered_gpt_data)

# Define the function to read the data
def read_data():
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)
    return data 
#get_line_data(get_date_ranges())
#get_accident_data(get_date_ranges())
#get_gpt_data(get_date_ranges())


# 2022-08-16 hypo-1 tem-0.7

In [10]:
get_gpt_data(get_date_ranges())

Start: 2018-08-15 00:00:00, End: 2018-08-16 00:00:00
[{"title": "U3 : Schadhaftes Fahrzeug", "description": "Wegen eines schadhaften Fahrzeuges werden die Z\u00c3\u0192\u00c2\u00bcge der Linie U3 zwischen den Stationen Zieglergasse und Volkstheater U \u00c3\u0192\u00c2\u00bcber Gleis 1 gef\u00c3\u0192\u00c2\u00bchrt.. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 23:45 Uhr!"}, {"title": "29A : Polizeieinsatz", "description": "Nach einer Fahrtbehinderung kommt es auf der Linie 29A zu unterschiedlichen Intervallen."}, {"title": "9 : Falschparker", "description": "Wegen eines Falschparkers im Bereich Hormayrgasse 18 ist die Linie 9 in Fahrtrichtung Wallri\u00c3\u0192\u00c2\u0178stra\u00c3\u0192\u00c2\u0178e an der Weiterfahrt gehindert. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 21:35 Uhr!"}, {"title": "13A :", "description": "Nach einer Fahrtbehinderung kommt es auf der Linie 13A zu unterschiedlichen Intervallen."}, {"title": "U1 : Polizeieinsatz", "des

In [11]:
import json
from dotenv import load_dotenv
from datetime import datetime, timedelta
import openai
import os
from pyecharts import *
#from plot import plot_line_data_to_html, plot_accident_data_to_html, plot_gpt_data_to_html 

load_dotenv()
# Configure the OpenAI API client
#openai.api_type = os.getenv("OPENAI_API_TYPE")
#openai.api_version = os.getenv("OPENAI_API_VERSION")
#openai.api_base = os.getenv("OPENAI_API_BASE")  # Your Azure OpenAI resource's endpoint value.
#openai.api_key = os.getenv("OPENAI_API_KEY")

# Define the function to generate the response
#def generate_response(resulted_data):
#    response = openai.ChatCompletion.create(
#        model="gpt-4",
        #engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
#        temperature = 0.4,
#        messages=[
#            {"role": "system", "content": "Du bist ein Analyse-Chatbot. Aus den bereitgestellten JSON-Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-JSON-Daten:"+ resulted_data},
#            {"role": "user", "content": "Im JSON-Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
#            }
#        ]
#    )
#    return response['choices'][0]['message']['content']

def generate_response(resulted_data):
    response = openai.ChatCompletion.create(
    model="gpt-4-0314",
    # engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
    temperature=0.5,
    messages=[
      {"role": "user",
       "content": "Du bist ein Analyst. Aus den bereitgestellten Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-List-Daten:" + resulted_data + "Im Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
      }
    ]
  )
    return response['choices'][0]['message']['content']


# Define the function to filter the accident data
def get_incident_data(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    #gpt filter
    filtered_incidents = [
        {
            'title': entry['title'],
            'description': entry['description']
        } 
        for entry in data['incidents'] 
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime
    ]
        
    resulted_data = json.dumps(filtered_incidents)
    
    print(resulted_data)
    
    #filtered_gpt_data = ''
    filtered_gpt_data = generate_response(resulted_data)
    
    return filtered_gpt_data



# Define the function to filter the line data 
def get_top10_lines(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    filtered_lines_incidents = []
    for entry in data['incidents']:
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime:
            filtered_lines_incidents.append(entry['lines'])

    from collections import Counter


    # Flatten the list of lists
    flattened_list = [item for sublist in filtered_lines_incidents for item in sublist]
    # Calculate the frequency of each element
    element_frequencies = Counter(flattened_list)
    # Get the top 10 most common elements and their frequencies
    top_10 = element_frequencies.most_common(10)

    return top_10

# Define the function to filter the accident time data
def get_accident_time_sum(data, start_time, end_time):
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)

    filtered_accident_sum = {"Verkehrsunfall": 0, "Falschparker": 0}
    #for entry in data['incidents']:
    for entry in data['incidents']:
        # Data validation for the 'start' and 'end' timestamps
        if 'start' not in entry or 'end' not in entry:
            #print("Missing 'start' or 'end' in entry:", entry)
            continue
        if not entry['start'] or not entry['end']:
            #print("Empty 'start' or 'end' timestamp in entry:", entry)
            continue

        try:
            start_incident_datetime = datetime.strptime(entry['start'], date_format)
            end_incident_datetime = datetime.strptime(entry['end'], date_format)
        except ValueError as e:
            #print(f"Error parsing timestamp in entry {entry}: {e}")
            continue

        # Check the incident title and compute the time sum
        if start_datetime <= start_incident_datetime <= end_datetime and ("Verkehrsunfall" in entry['title'] or "Falschparker" in entry['title']):
            if "Verkehrsunfall" in entry['title']:
                filtered_accident_sum['Verkehrsunfall'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60
            if "Falschparker" in entry['title']:
                filtered_accident_sum['Falschparker'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60

    filtered_accident_sum = {key: (round(value, 2) if isinstance(value, float) else value) for key, value in filtered_accident_sum.items()}

    return filtered_accident_sum


# Define the function to generate the date ranges
def get_date_ranges():
    # Initialize the end date to today's date at 00:00:00
    end_time = datetime.strptime('2018-08-16 00:00:00.00000','%Y-%m-%d %H:%M:%S.%f')
    # Define the time interval (1 week = 7 days)
    interval = timedelta(days=1)
    date_ranges = []
    # Generate the ranges
    for _ in range(3):
        start_time = end_time - interval
        date_ranges.append((start_time, end_time))
        end_time = start_time
    return date_ranges


# Define the function to generate the Subway_line data
def get_line_data(date_ranges):
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)

    #top 10 line data
    lines_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filetered_top_10_line_data = get_top10_lines(data, start_str, end_str)
        #print("Line data from data analysis:")
        lines_data_list.append(filetered_top_10_line_data)
    print(lines_data_list)
    return plot_line_data_to_html(lines_data_list)


# Define the function to generate the accident data
def get_accident_data(date_ranges):
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)

    #accident time sum daata
    accident_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filtered_accident_time_data = get_accident_time_sum(data, start_str, end_str)
        #print("Filtered Accident time data")
        accident_data_list.append(filtered_accident_time_data)
    print(accident_data_list)
    return plot_accident_data_to_html(accident_data_list)


# Define the function to generate the gpt data
def get_gpt_data(date_ranges):
    # Use the list of tuples in a for loop
    data = read_data()
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        
        print(f"Start: {start_str}, End: {end_str}")
        filtered_gpt_data = get_incident_data(data, start_str, end_str)
        
        print("Filtered GPT station data:")
        print(filtered_gpt_data)
        break
    return plot_gpt_data_to_html(filtered_gpt_data)

# Define the function to read the data
def read_data():
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)
    return data 
#get_line_data(get_date_ranges())
#get_accident_data(get_date_ranges())
#get_gpt_data(get_date_ranges())


# 2022-08-16 hypo-1 tem-0.5

In [12]:
get_gpt_data(get_date_ranges())  

Start: 2018-08-15 00:00:00, End: 2018-08-16 00:00:00
[{"title": "U3 : Schadhaftes Fahrzeug", "description": "Wegen eines schadhaften Fahrzeuges werden die Z\u00c3\u0192\u00c2\u00bcge der Linie U3 zwischen den Stationen Zieglergasse und Volkstheater U \u00c3\u0192\u00c2\u00bcber Gleis 1 gef\u00c3\u0192\u00c2\u00bchrt.. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 23:45 Uhr!"}, {"title": "29A : Polizeieinsatz", "description": "Nach einer Fahrtbehinderung kommt es auf der Linie 29A zu unterschiedlichen Intervallen."}, {"title": "9 : Falschparker", "description": "Wegen eines Falschparkers im Bereich Hormayrgasse 18 ist die Linie 9 in Fahrtrichtung Wallri\u00c3\u0192\u00c2\u0178stra\u00c3\u0192\u00c2\u0178e an der Weiterfahrt gehindert. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 21:35 Uhr!"}, {"title": "13A :", "description": "Nach einer Fahrtbehinderung kommt es auf der Linie 13A zu unterschiedlichen Intervallen."}, {"title": "U1 : Polizeieinsatz", "des

In [13]:
import json
from dotenv import load_dotenv
from datetime import datetime, timedelta
import openai
import os
from pyecharts import *
#from plot import plot_line_data_to_html, plot_accident_data_to_html, plot_gpt_data_to_html 

load_dotenv()
# Configure the OpenAI API client
#openai.api_type = os.getenv("OPENAI_API_TYPE")
#openai.api_version = os.getenv("OPENAI_API_VERSION")
#openai.api_base = os.getenv("OPENAI_API_BASE")  # Your Azure OpenAI resource's endpoint value.
#openai.api_key = os.getenv("OPENAI_API_KEY")

# Define the function to generate the response
#def generate_response(resulted_data):
#    response = openai.ChatCompletion.create(
#        model="gpt-4",
        #engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
#        temperature = 0.4,
#        messages=[
#            {"role": "system", "content": "Du bist ein Analyse-Chatbot. Aus den bereitgestellten JSON-Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-JSON-Daten:"+ resulted_data},
#            {"role": "user", "content": "Im JSON-Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
#            }
#        ]
#    )
#    return response['choices'][0]['message']['content']

def generate_response(resulted_data):
    response = openai.ChatCompletion.create(
    model="gpt-4-0314",
    # engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
    temperature=0.6,
    messages=[
      {"role": "user",
       "content": "Du bist ein Analyst. Aus den bereitgestellten Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-List-Daten:" + resulted_data + "Im Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
      }
    ]
  )
    return response['choices'][0]['message']['content']


# Define the function to filter the accident data
def get_incident_data(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    #gpt filter
    filtered_incidents = [
        {
            'title': entry['title'],
            'description': entry['description']
        } 
        for entry in data['incidents'] 
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime
    ]
        
    resulted_data = json.dumps(filtered_incidents)
    
    print(resulted_data)
    
    #filtered_gpt_data = ''
    filtered_gpt_data = generate_response(resulted_data)
    
    return filtered_gpt_data



# Define the function to filter the line data 
def get_top10_lines(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    filtered_lines_incidents = []
    for entry in data['incidents']:
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime:
            filtered_lines_incidents.append(entry['lines'])

    from collections import Counter


    # Flatten the list of lists
    flattened_list = [item for sublist in filtered_lines_incidents for item in sublist]
    # Calculate the frequency of each element
    element_frequencies = Counter(flattened_list)
    # Get the top 10 most common elements and their frequencies
    top_10 = element_frequencies.most_common(10)

    return top_10

# Define the function to filter the accident time data
def get_accident_time_sum(data, start_time, end_time):
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)

    filtered_accident_sum = {"Verkehrsunfall": 0, "Falschparker": 0}
    #for entry in data['incidents']:
    for entry in data['incidents']:
        # Data validation for the 'start' and 'end' timestamps
        if 'start' not in entry or 'end' not in entry:
            #print("Missing 'start' or 'end' in entry:", entry)
            continue
        if not entry['start'] or not entry['end']:
            #print("Empty 'start' or 'end' timestamp in entry:", entry)
            continue

        try:
            start_incident_datetime = datetime.strptime(entry['start'], date_format)
            end_incident_datetime = datetime.strptime(entry['end'], date_format)
        except ValueError as e:
            #print(f"Error parsing timestamp in entry {entry}: {e}")
            continue

        # Check the incident title and compute the time sum
        if start_datetime <= start_incident_datetime <= end_datetime and ("Verkehrsunfall" in entry['title'] or "Falschparker" in entry['title']):
            if "Verkehrsunfall" in entry['title']:
                filtered_accident_sum['Verkehrsunfall'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60
            if "Falschparker" in entry['title']:
                filtered_accident_sum['Falschparker'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60

    filtered_accident_sum = {key: (round(value, 2) if isinstance(value, float) else value) for key, value in filtered_accident_sum.items()}

    return filtered_accident_sum


# Define the function to generate the date ranges
def get_date_ranges():
    # Initialize the end date to today's date at 00:00:00
    end_time = datetime.strptime('2018-08-16 00:00:00.00000','%Y-%m-%d %H:%M:%S.%f')
    # Define the time interval (1 week = 7 days)
    interval = timedelta(days=1)
    date_ranges = []
    # Generate the ranges
    for _ in range(3):
        start_time = end_time - interval
        date_ranges.append((start_time, end_time))
        end_time = start_time
    return date_ranges


# Define the function to generate the Subway_line data
def get_line_data(date_ranges):
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)

    #top 10 line data
    lines_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filetered_top_10_line_data = get_top10_lines(data, start_str, end_str)
        #print("Line data from data analysis:")
        lines_data_list.append(filetered_top_10_line_data)
    print(lines_data_list)
    return plot_line_data_to_html(lines_data_list)


# Define the function to generate the accident data
def get_accident_data(date_ranges):
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)

    #accident time sum daata
    accident_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filtered_accident_time_data = get_accident_time_sum(data, start_str, end_str)
        #print("Filtered Accident time data")
        accident_data_list.append(filtered_accident_time_data)
    print(accident_data_list)
    return plot_accident_data_to_html(accident_data_list)


# Define the function to generate the gpt data
def get_gpt_data(date_ranges):
    # Use the list of tuples in a for loop
    data = read_data()
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        
        print(f"Start: {start_str}, End: {end_str}")
        filtered_gpt_data = get_incident_data(data, start_str, end_str)
        
        print("Filtered GPT station data:")
        print(filtered_gpt_data)
        break
    return plot_gpt_data_to_html(filtered_gpt_data)

# Define the function to read the data
def read_data():
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)
    return data 
#get_line_data(get_date_ranges())
#get_accident_data(get_date_ranges())
#get_gpt_data(get_date_ranges())


# 2022-08-16 hypo-1 tem-0.6

In [14]:
get_gpt_data(get_date_ranges()) 

Start: 2018-08-15 00:00:00, End: 2018-08-16 00:00:00
[{"title": "U3 : Schadhaftes Fahrzeug", "description": "Wegen eines schadhaften Fahrzeuges werden die Z\u00c3\u0192\u00c2\u00bcge der Linie U3 zwischen den Stationen Zieglergasse und Volkstheater U \u00c3\u0192\u00c2\u00bcber Gleis 1 gef\u00c3\u0192\u00c2\u00bchrt.. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 23:45 Uhr!"}, {"title": "29A : Polizeieinsatz", "description": "Nach einer Fahrtbehinderung kommt es auf der Linie 29A zu unterschiedlichen Intervallen."}, {"title": "9 : Falschparker", "description": "Wegen eines Falschparkers im Bereich Hormayrgasse 18 ist die Linie 9 in Fahrtrichtung Wallri\u00c3\u0192\u00c2\u0178stra\u00c3\u0192\u00c2\u0178e an der Weiterfahrt gehindert. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 21:35 Uhr!"}, {"title": "13A :", "description": "Nach einer Fahrtbehinderung kommt es auf der Linie 13A zu unterschiedlichen Intervallen."}, {"title": "U1 : Polizeieinsatz", "des

In [15]:
import json
from dotenv import load_dotenv
from datetime import datetime, timedelta
import openai
import os
from pyecharts import *
#from plot import plot_line_data_to_html, plot_accident_data_to_html, plot_gpt_data_to_html 

load_dotenv()
# Configure the OpenAI API client
#openai.api_type = os.getenv("OPENAI_API_TYPE")
#openai.api_version = os.getenv("OPENAI_API_VERSION")
#openai.api_base = os.getenv("OPENAI_API_BASE")  # Your Azure OpenAI resource's endpoint value.
#openai.api_key = os.getenv("OPENAI_API_KEY")

# Define the function to generate the response
#def generate_response(resulted_data):
#    response = openai.ChatCompletion.create(
#        model="gpt-4",
        #engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
#        temperature = 0.4,
#        messages=[
#            {"role": "system", "content": "Du bist ein Analyse-Chatbot. Aus den bereitgestellten JSON-Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-JSON-Daten:"+ resulted_data},
#            {"role": "user", "content": "Im JSON-Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
#            }
#        ]
#    )
#    return response['choices'][0]['message']['content']

def generate_response(resulted_data):
    response = openai.ChatCompletion.create(
    model="gpt-4-0314",
    # engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
    temperature=0.4,
    messages=[
      {"role": "user",
       "content": "Du bist ein Analyst. Aus den bereitgestellten Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-List-Daten:" + resulted_data + "Im Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
      }
    ]
  )
    return response['choices'][0]['message']['content']


# Define the function to filter the accident data
def get_incident_data(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    #gpt filter
    filtered_incidents = [
        {
            'title': entry['title'],
            'description': entry['description']
        } 
        for entry in data['incidents'] 
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime
    ]
        
    resulted_data = json.dumps(filtered_incidents)
    
    print(resulted_data)
    
    #filtered_gpt_data = ''
    filtered_gpt_data = generate_response(resulted_data)
    
    return filtered_gpt_data



# Define the function to filter the line data 
def get_top10_lines(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    filtered_lines_incidents = []
    for entry in data['incidents']:
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime:
            filtered_lines_incidents.append(entry['lines'])

    from collections import Counter


    # Flatten the list of lists
    flattened_list = [item for sublist in filtered_lines_incidents for item in sublist]
    # Calculate the frequency of each element
    element_frequencies = Counter(flattened_list)
    # Get the top 10 most common elements and their frequencies
    top_10 = element_frequencies.most_common(10)

    return top_10

# Define the function to filter the accident time data
def get_accident_time_sum(data, start_time, end_time):
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)

    filtered_accident_sum = {"Verkehrsunfall": 0, "Falschparker": 0}
    #for entry in data['incidents']:
    for entry in data['incidents']:
        # Data validation for the 'start' and 'end' timestamps
        if 'start' not in entry or 'end' not in entry:
            #print("Missing 'start' or 'end' in entry:", entry)
            continue
        if not entry['start'] or not entry['end']:
            #print("Empty 'start' or 'end' timestamp in entry:", entry)
            continue

        try:
            start_incident_datetime = datetime.strptime(entry['start'], date_format)
            end_incident_datetime = datetime.strptime(entry['end'], date_format)
        except ValueError as e:
            #print(f"Error parsing timestamp in entry {entry}: {e}")
            continue

        # Check the incident title and compute the time sum
        if start_datetime <= start_incident_datetime <= end_datetime and ("Verkehrsunfall" in entry['title'] or "Falschparker" in entry['title']):
            if "Verkehrsunfall" in entry['title']:
                filtered_accident_sum['Verkehrsunfall'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60
            if "Falschparker" in entry['title']:
                filtered_accident_sum['Falschparker'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60

    filtered_accident_sum = {key: (round(value, 2) if isinstance(value, float) else value) for key, value in filtered_accident_sum.items()}

    return filtered_accident_sum


# Define the function to generate the date ranges
def get_date_ranges():
    # Initialize the end date to today's date at 00:00:00
    end_time = datetime.strptime('2018-08-16 00:00:00.00000','%Y-%m-%d %H:%M:%S.%f')
    # Define the time interval (1 week = 7 days)
    interval = timedelta(days=1)
    date_ranges = []
    # Generate the ranges
    for _ in range(3):
        start_time = end_time - interval
        date_ranges.append((start_time, end_time))
        end_time = start_time
    return date_ranges


# Define the function to generate the Subway_line data
def get_line_data(date_ranges):
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)

    #top 10 line data
    lines_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filetered_top_10_line_data = get_top10_lines(data, start_str, end_str)
        #print("Line data from data analysis:")
        lines_data_list.append(filetered_top_10_line_data)
    print(lines_data_list)
    return plot_line_data_to_html(lines_data_list)


# Define the function to generate the accident data
def get_accident_data(date_ranges):
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)

    #accident time sum daata
    accident_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filtered_accident_time_data = get_accident_time_sum(data, start_str, end_str)
        #print("Filtered Accident time data")
        accident_data_list.append(filtered_accident_time_data)
    print(accident_data_list)
    return plot_accident_data_to_html(accident_data_list)


# Define the function to generate the gpt data
def get_gpt_data(date_ranges):
    # Use the list of tuples in a for loop
    data = read_data()
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        
        print(f"Start: {start_str}, End: {end_str}")
        filtered_gpt_data = get_incident_data(data, start_str, end_str)
        
        print("Filtered GPT station data:")
        print(filtered_gpt_data)
        break
    return plot_gpt_data_to_html(filtered_gpt_data)

# Define the function to read the data
def read_data():
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)
    return data 
#get_line_data(get_date_ranges())
#get_accident_data(get_date_ranges())
#get_gpt_data(get_date_ranges())


# 2022-08-16 hypo-1 tem-0.4

In [16]:
get_gpt_data(get_date_ranges())

Start: 2018-08-15 00:00:00, End: 2018-08-16 00:00:00
[{"title": "U3 : Schadhaftes Fahrzeug", "description": "Wegen eines schadhaften Fahrzeuges werden die Z\u00c3\u0192\u00c2\u00bcge der Linie U3 zwischen den Stationen Zieglergasse und Volkstheater U \u00c3\u0192\u00c2\u00bcber Gleis 1 gef\u00c3\u0192\u00c2\u00bchrt.. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 23:45 Uhr!"}, {"title": "29A : Polizeieinsatz", "description": "Nach einer Fahrtbehinderung kommt es auf der Linie 29A zu unterschiedlichen Intervallen."}, {"title": "9 : Falschparker", "description": "Wegen eines Falschparkers im Bereich Hormayrgasse 18 ist die Linie 9 in Fahrtrichtung Wallri\u00c3\u0192\u00c2\u0178stra\u00c3\u0192\u00c2\u0178e an der Weiterfahrt gehindert. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 21:35 Uhr!"}, {"title": "13A :", "description": "Nach einer Fahrtbehinderung kommt es auf der Linie 13A zu unterschiedlichen Intervallen."}, {"title": "U1 : Polizeieinsatz", "des

In [17]:
import json
from dotenv import load_dotenv
from datetime import datetime, timedelta
import openai
import os
from pyecharts import *
#from plot import plot_line_data_to_html, plot_accident_data_to_html, plot_gpt_data_to_html 

load_dotenv()
# Configure the OpenAI API client
#openai.api_type = os.getenv("OPENAI_API_TYPE")
#openai.api_version = os.getenv("OPENAI_API_VERSION")
#openai.api_base = os.getenv("OPENAI_API_BASE")  # Your Azure OpenAI resource's endpoint value.
#openai.api_key = os.getenv("OPENAI_API_KEY")

# Define the function to generate the response
#def generate_response(resulted_data):
#    response = openai.ChatCompletion.create(
#        model="gpt-4",
        #engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
#        temperature = 0.4,
#        messages=[
#            {"role": "system", "content": "Du bist ein Analyse-Chatbot. Aus den bereitgestellten JSON-Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-JSON-Daten:"+ resulted_data},
#            {"role": "user", "content": "Im JSON-Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
#            }
#        ]
#    )
#    return response['choices'][0]['message']['content']

def generate_response(resulted_data):
    response = openai.ChatCompletion.create(
    model="gpt-4-0314",
    # engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
    temperature=0.3,
    messages=[
      {"role": "user",
       "content": "Du bist ein Analyst. Aus den bereitgestellten Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-List-Daten:" + resulted_data + "Im Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
      }
    ]
  )
    return response['choices'][0]['message']['content']


# Define the function to filter the accident data
def get_incident_data(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    #gpt filter
    filtered_incidents = [
        {
            'title': entry['title'],
            'description': entry['description']
        } 
        for entry in data['incidents'] 
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime
    ]
        
    resulted_data = json.dumps(filtered_incidents)
    
    print(resulted_data)
    
    #filtered_gpt_data = ''
    filtered_gpt_data = generate_response(resulted_data)
    
    return filtered_gpt_data



# Define the function to filter the line data 
def get_top10_lines(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    filtered_lines_incidents = []
    for entry in data['incidents']:
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime:
            filtered_lines_incidents.append(entry['lines'])

    from collections import Counter


    # Flatten the list of lists
    flattened_list = [item for sublist in filtered_lines_incidents for item in sublist]
    # Calculate the frequency of each element
    element_frequencies = Counter(flattened_list)
    # Get the top 10 most common elements and their frequencies
    top_10 = element_frequencies.most_common(10)

    return top_10

# Define the function to filter the accident time data
def get_accident_time_sum(data, start_time, end_time):
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)

    filtered_accident_sum = {"Verkehrsunfall": 0, "Falschparker": 0}
    #for entry in data['incidents']:
    for entry in data['incidents']:
        # Data validation for the 'start' and 'end' timestamps
        if 'start' not in entry or 'end' not in entry:
            #print("Missing 'start' or 'end' in entry:", entry)
            continue
        if not entry['start'] or not entry['end']:
            #print("Empty 'start' or 'end' timestamp in entry:", entry)
            continue

        try:
            start_incident_datetime = datetime.strptime(entry['start'], date_format)
            end_incident_datetime = datetime.strptime(entry['end'], date_format)
        except ValueError as e:
            #print(f"Error parsing timestamp in entry {entry}: {e}")
            continue

        # Check the incident title and compute the time sum
        if start_datetime <= start_incident_datetime <= end_datetime and ("Verkehrsunfall" in entry['title'] or "Falschparker" in entry['title']):
            if "Verkehrsunfall" in entry['title']:
                filtered_accident_sum['Verkehrsunfall'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60
            if "Falschparker" in entry['title']:
                filtered_accident_sum['Falschparker'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60

    filtered_accident_sum = {key: (round(value, 2) if isinstance(value, float) else value) for key, value in filtered_accident_sum.items()}

    return filtered_accident_sum


# Define the function to generate the date ranges
def get_date_ranges():
    # Initialize the end date to today's date at 00:00:00
    end_time = datetime.strptime('2018-08-16 00:00:00.00000','%Y-%m-%d %H:%M:%S.%f')
    # Define the time interval (1 week = 7 days)
    interval = timedelta(days=1)
    date_ranges = []
    # Generate the ranges
    for _ in range(3):
        start_time = end_time - interval
        date_ranges.append((start_time, end_time))
        end_time = start_time
    return date_ranges


# Define the function to generate the Subway_line data
def get_line_data(date_ranges):
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)

    #top 10 line data
    lines_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filetered_top_10_line_data = get_top10_lines(data, start_str, end_str)
        #print("Line data from data analysis:")
        lines_data_list.append(filetered_top_10_line_data)
    print(lines_data_list)
    return plot_line_data_to_html(lines_data_list)


# Define the function to generate the accident data
def get_accident_data(date_ranges):
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)

    #accident time sum daata
    accident_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filtered_accident_time_data = get_accident_time_sum(data, start_str, end_str)
        #print("Filtered Accident time data")
        accident_data_list.append(filtered_accident_time_data)
    print(accident_data_list)
    return plot_accident_data_to_html(accident_data_list)


# Define the function to generate the gpt data
def get_gpt_data(date_ranges):
    # Use the list of tuples in a for loop
    data = read_data()
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        
        print(f"Start: {start_str}, End: {end_str}")
        filtered_gpt_data = get_incident_data(data, start_str, end_str)
        
        print("Filtered GPT station data:")
        print(filtered_gpt_data)
        break
    return plot_gpt_data_to_html(filtered_gpt_data)

# Define the function to read the data
def read_data():
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)
    return data 
#get_line_data(get_date_ranges())
#get_accident_data(get_date_ranges())
#get_gpt_data(get_date_ranges())


# 2022-08-16 hypo-1 tem-0.3

In [18]:
get_gpt_data(get_date_ranges())

Start: 2018-08-15 00:00:00, End: 2018-08-16 00:00:00
[{"title": "U3 : Schadhaftes Fahrzeug", "description": "Wegen eines schadhaften Fahrzeuges werden die Z\u00c3\u0192\u00c2\u00bcge der Linie U3 zwischen den Stationen Zieglergasse und Volkstheater U \u00c3\u0192\u00c2\u00bcber Gleis 1 gef\u00c3\u0192\u00c2\u00bchrt.. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 23:45 Uhr!"}, {"title": "29A : Polizeieinsatz", "description": "Nach einer Fahrtbehinderung kommt es auf der Linie 29A zu unterschiedlichen Intervallen."}, {"title": "9 : Falschparker", "description": "Wegen eines Falschparkers im Bereich Hormayrgasse 18 ist die Linie 9 in Fahrtrichtung Wallri\u00c3\u0192\u00c2\u0178stra\u00c3\u0192\u00c2\u0178e an der Weiterfahrt gehindert. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 21:35 Uhr!"}, {"title": "13A :", "description": "Nach einer Fahrtbehinderung kommt es auf der Linie 13A zu unterschiedlichen Intervallen."}, {"title": "U1 : Polizeieinsatz", "des

In [19]:
import json
from dotenv import load_dotenv
from datetime import datetime, timedelta
import openai
import os
from pyecharts import *
#from plot import plot_line_data_to_html, plot_accident_data_to_html, plot_gpt_data_to_html 

load_dotenv()
# Configure the OpenAI API client
#openai.api_type = os.getenv("OPENAI_API_TYPE")
#openai.api_version = os.getenv("OPENAI_API_VERSION")
#openai.api_base = os.getenv("OPENAI_API_BASE")  # Your Azure OpenAI resource's endpoint value.
#openai.api_key = os.getenv("OPENAI_API_KEY")

# Define the function to generate the response
#def generate_response(resulted_data):
#    response = openai.ChatCompletion.create(
#        model="gpt-4",
        #engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
#        temperature = 0.4,
#        messages=[
#            {"role": "system", "content": "Du bist ein Analyse-Chatbot. Aus den bereitgestellten JSON-Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-JSON-Daten:"+ resulted_data},
#            {"role": "user", "content": "Im JSON-Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
#            }
#        ]
#    )
#    return response['choices'][0]['message']['content']

def generate_response(resulted_data):
    response = openai.ChatCompletion.create(
    model="gpt-4-0314",
    # engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
    temperature=0.2,
    messages=[
      {"role": "user",
       "content": "Du bist ein Analyst. Aus den bereitgestellten Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-List-Daten:" + resulted_data + "Im Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
      }
    ]
  )
    return response['choices'][0]['message']['content']


# Define the function to filter the accident data
def get_incident_data(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    #gpt filter
    filtered_incidents = [
        {
            'title': entry['title'],
            'description': entry['description']
        } 
        for entry in data['incidents'] 
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime
    ]
        
    resulted_data = json.dumps(filtered_incidents)
    
    print(resulted_data)
    
    #filtered_gpt_data = ''
    filtered_gpt_data = generate_response(resulted_data)
    
    return filtered_gpt_data



# Define the function to filter the line data 
def get_top10_lines(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    filtered_lines_incidents = []
    for entry in data['incidents']:
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime:
            filtered_lines_incidents.append(entry['lines'])

    from collections import Counter


    # Flatten the list of lists
    flattened_list = [item for sublist in filtered_lines_incidents for item in sublist]
    # Calculate the frequency of each element
    element_frequencies = Counter(flattened_list)
    # Get the top 10 most common elements and their frequencies
    top_10 = element_frequencies.most_common(10)

    return top_10

# Define the function to filter the accident time data
def get_accident_time_sum(data, start_time, end_time):
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)

    filtered_accident_sum = {"Verkehrsunfall": 0, "Falschparker": 0}
    #for entry in data['incidents']:
    for entry in data['incidents']:
        # Data validation for the 'start' and 'end' timestamps
        if 'start' not in entry or 'end' not in entry:
            #print("Missing 'start' or 'end' in entry:", entry)
            continue
        if not entry['start'] or not entry['end']:
            #print("Empty 'start' or 'end' timestamp in entry:", entry)
            continue

        try:
            start_incident_datetime = datetime.strptime(entry['start'], date_format)
            end_incident_datetime = datetime.strptime(entry['end'], date_format)
        except ValueError as e:
            #print(f"Error parsing timestamp in entry {entry}: {e}")
            continue

        # Check the incident title and compute the time sum
        if start_datetime <= start_incident_datetime <= end_datetime and ("Verkehrsunfall" in entry['title'] or "Falschparker" in entry['title']):
            if "Verkehrsunfall" in entry['title']:
                filtered_accident_sum['Verkehrsunfall'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60
            if "Falschparker" in entry['title']:
                filtered_accident_sum['Falschparker'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60

    filtered_accident_sum = {key: (round(value, 2) if isinstance(value, float) else value) for key, value in filtered_accident_sum.items()}

    return filtered_accident_sum


# Define the function to generate the date ranges
def get_date_ranges():
    # Initialize the end date to today's date at 00:00:00
    end_time = datetime.strptime('2018-08-16 00:00:00.00000','%Y-%m-%d %H:%M:%S.%f')
    # Define the time interval (1 week = 7 days)
    interval = timedelta(days=1)
    date_ranges = []
    # Generate the ranges
    for _ in range(3):
        start_time = end_time - interval
        date_ranges.append((start_time, end_time))
        end_time = start_time
    return date_ranges


# Define the function to generate the Subway_line data
def get_line_data(date_ranges):
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)

    #top 10 line data
    lines_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filetered_top_10_line_data = get_top10_lines(data, start_str, end_str)
        #print("Line data from data analysis:")
        lines_data_list.append(filetered_top_10_line_data)
    print(lines_data_list)
    return plot_line_data_to_html(lines_data_list)


# Define the function to generate the accident data
def get_accident_data(date_ranges):
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)

    #accident time sum daata
    accident_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filtered_accident_time_data = get_accident_time_sum(data, start_str, end_str)
        #print("Filtered Accident time data")
        accident_data_list.append(filtered_accident_time_data)
    print(accident_data_list)
    return plot_accident_data_to_html(accident_data_list)


# Define the function to generate the gpt data
def get_gpt_data(date_ranges):
    # Use the list of tuples in a for loop
    data = read_data()
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        
        print(f"Start: {start_str}, End: {end_str}")
        filtered_gpt_data = get_incident_data(data, start_str, end_str)
        
        print("Filtered GPT station data:")
        print(filtered_gpt_data)
        break
    return plot_gpt_data_to_html(filtered_gpt_data)

# Define the function to read the data
def read_data():
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)
    return data 
#get_line_data(get_date_ranges())
#get_accident_data(get_date_ranges())
#get_gpt_data(get_date_ranges())


# 2022-08-16 hypo-1 tem-0.2

In [20]:
get_gpt_data(get_date_ranges())

Start: 2018-08-15 00:00:00, End: 2018-08-16 00:00:00
[{"title": "U3 : Schadhaftes Fahrzeug", "description": "Wegen eines schadhaften Fahrzeuges werden die Z\u00c3\u0192\u00c2\u00bcge der Linie U3 zwischen den Stationen Zieglergasse und Volkstheater U \u00c3\u0192\u00c2\u00bcber Gleis 1 gef\u00c3\u0192\u00c2\u00bchrt.. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 23:45 Uhr!"}, {"title": "29A : Polizeieinsatz", "description": "Nach einer Fahrtbehinderung kommt es auf der Linie 29A zu unterschiedlichen Intervallen."}, {"title": "9 : Falschparker", "description": "Wegen eines Falschparkers im Bereich Hormayrgasse 18 ist die Linie 9 in Fahrtrichtung Wallri\u00c3\u0192\u00c2\u0178stra\u00c3\u0192\u00c2\u0178e an der Weiterfahrt gehindert. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 21:35 Uhr!"}, {"title": "13A :", "description": "Nach einer Fahrtbehinderung kommt es auf der Linie 13A zu unterschiedlichen Intervallen."}, {"title": "U1 : Polizeieinsatz", "des

In [21]:
import json
from dotenv import load_dotenv
from datetime import datetime, timedelta
import openai
import os
from pyecharts import *
#from plot import plot_line_data_to_html, plot_accident_data_to_html, plot_gpt_data_to_html 

load_dotenv()
# Configure the OpenAI API client
#openai.api_type = os.getenv("OPENAI_API_TYPE")
#openai.api_version = os.getenv("OPENAI_API_VERSION")
#openai.api_base = os.getenv("OPENAI_API_BASE")  # Your Azure OpenAI resource's endpoint value.
#openai.api_key = os.getenv("OPENAI_API_KEY")

# Define the function to generate the response
#def generate_response(resulted_data):
#    response = openai.ChatCompletion.create(
#        model="gpt-4",
        #engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
#        temperature = 0.4,
#        messages=[
#            {"role": "system", "content": "Du bist ein Analyse-Chatbot. Aus den bereitgestellten JSON-Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-JSON-Daten:"+ resulted_data},
#            {"role": "user", "content": "Im JSON-Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
#            }
#        ]
#    )
#    return response['choices'][0]['message']['content']

def generate_response(resulted_data):
    response = openai.ChatCompletion.create(
    model="gpt-4-0314",
    # engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
    temperature=0.1,
    messages=[
      {"role": "user",
       "content": "Du bist ein Analyst. Aus den bereitgestellten Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-List-Daten:" + resulted_data + "Im Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
      }
    ]
  )
    return response['choices'][0]['message']['content']


# Define the function to filter the accident data
def get_incident_data(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    #gpt filter
    filtered_incidents = [
        {
            'title': entry['title'],
            'description': entry['description']
        } 
        for entry in data['incidents'] 
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime
    ]
        
    resulted_data = json.dumps(filtered_incidents)
    
    print(resulted_data)
    
    #filtered_gpt_data = ''
    filtered_gpt_data = generate_response(resulted_data)
    
    return filtered_gpt_data



# Define the function to filter the line data 
def get_top10_lines(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    filtered_lines_incidents = []
    for entry in data['incidents']:
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime:
            filtered_lines_incidents.append(entry['lines'])

    from collections import Counter


    # Flatten the list of lists
    flattened_list = [item for sublist in filtered_lines_incidents for item in sublist]
    # Calculate the frequency of each element
    element_frequencies = Counter(flattened_list)
    # Get the top 10 most common elements and their frequencies
    top_10 = element_frequencies.most_common(10)

    return top_10

# Define the function to filter the accident time data
def get_accident_time_sum(data, start_time, end_time):
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)

    filtered_accident_sum = {"Verkehrsunfall": 0, "Falschparker": 0}
    #for entry in data['incidents']:
    for entry in data['incidents']:
        # Data validation for the 'start' and 'end' timestamps
        if 'start' not in entry or 'end' not in entry:
            #print("Missing 'start' or 'end' in entry:", entry)
            continue
        if not entry['start'] or not entry['end']:
            #print("Empty 'start' or 'end' timestamp in entry:", entry)
            continue

        try:
            start_incident_datetime = datetime.strptime(entry['start'], date_format)
            end_incident_datetime = datetime.strptime(entry['end'], date_format)
        except ValueError as e:
            #print(f"Error parsing timestamp in entry {entry}: {e}")
            continue

        # Check the incident title and compute the time sum
        if start_datetime <= start_incident_datetime <= end_datetime and ("Verkehrsunfall" in entry['title'] or "Falschparker" in entry['title']):
            if "Verkehrsunfall" in entry['title']:
                filtered_accident_sum['Verkehrsunfall'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60
            if "Falschparker" in entry['title']:
                filtered_accident_sum['Falschparker'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60

    filtered_accident_sum = {key: (round(value, 2) if isinstance(value, float) else value) for key, value in filtered_accident_sum.items()}

    return filtered_accident_sum


# Define the function to generate the date ranges
def get_date_ranges():
    # Initialize the end date to today's date at 00:00:00
    end_time = datetime.strptime('2018-08-16 00:00:00.00000','%Y-%m-%d %H:%M:%S.%f')
    # Define the time interval (1 week = 7 days)
    interval = timedelta(days=1)
    date_ranges = []
    # Generate the ranges
    for _ in range(3):
        start_time = end_time - interval
        date_ranges.append((start_time, end_time))
        end_time = start_time
    return date_ranges


# Define the function to generate the Subway_line data
def get_line_data(date_ranges):
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)

    #top 10 line data
    lines_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filetered_top_10_line_data = get_top10_lines(data, start_str, end_str)
        #print("Line data from data analysis:")
        lines_data_list.append(filetered_top_10_line_data)
    print(lines_data_list)
    return plot_line_data_to_html(lines_data_list)


# Define the function to generate the accident data
def get_accident_data(date_ranges):
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)

    #accident time sum daata
    accident_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filtered_accident_time_data = get_accident_time_sum(data, start_str, end_str)
        #print("Filtered Accident time data")
        accident_data_list.append(filtered_accident_time_data)
    print(accident_data_list)
    return plot_accident_data_to_html(accident_data_list)


# Define the function to generate the gpt data
def get_gpt_data(date_ranges):
    # Use the list of tuples in a for loop
    data = read_data()
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        
        print(f"Start: {start_str}, End: {end_str}")
        filtered_gpt_data = get_incident_data(data, start_str, end_str)
        
        print("Filtered GPT station data:")
        print(filtered_gpt_data)
        break
    return plot_gpt_data_to_html(filtered_gpt_data)

# Define the function to read the data
def read_data():
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)
    return data 
#get_line_data(get_date_ranges())
#get_accident_data(get_date_ranges())
#get_gpt_data(get_date_ranges())


# 2022-08-16 hypo-1 tem-0.1

In [22]:
get_gpt_data(get_date_ranges())

Start: 2018-08-15 00:00:00, End: 2018-08-16 00:00:00
[{"title": "U3 : Schadhaftes Fahrzeug", "description": "Wegen eines schadhaften Fahrzeuges werden die Z\u00c3\u0192\u00c2\u00bcge der Linie U3 zwischen den Stationen Zieglergasse und Volkstheater U \u00c3\u0192\u00c2\u00bcber Gleis 1 gef\u00c3\u0192\u00c2\u00bchrt.. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 23:45 Uhr!"}, {"title": "29A : Polizeieinsatz", "description": "Nach einer Fahrtbehinderung kommt es auf der Linie 29A zu unterschiedlichen Intervallen."}, {"title": "9 : Falschparker", "description": "Wegen eines Falschparkers im Bereich Hormayrgasse 18 ist die Linie 9 in Fahrtrichtung Wallri\u00c3\u0192\u00c2\u0178stra\u00c3\u0192\u00c2\u0178e an der Weiterfahrt gehindert. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 21:35 Uhr!"}, {"title": "13A :", "description": "Nach einer Fahrtbehinderung kommt es auf der Linie 13A zu unterschiedlichen Intervallen."}, {"title": "U1 : Polizeieinsatz", "des

In [23]:
import json
from dotenv import load_dotenv
from datetime import datetime, timedelta
import openai
import os
from pyecharts import *
#from plot import plot_line_data_to_html, plot_accident_data_to_html, plot_gpt_data_to_html 

load_dotenv()
# Configure the OpenAI API client
#openai.api_type = os.getenv("OPENAI_API_TYPE")
#openai.api_version = os.getenv("OPENAI_API_VERSION")
#openai.api_base = os.getenv("OPENAI_API_BASE")  # Your Azure OpenAI resource's endpoint value.
#openai.api_key = os.getenv("OPENAI_API_KEY")

# Define the function to generate the response
#def generate_response(resulted_data):
#    response = openai.ChatCompletion.create(
#        model="gpt-4",
        #engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
#        temperature = 0.4,
#        messages=[
#            {"role": "system", "content": "Du bist ein Analyse-Chatbot. Aus den bereitgestellten JSON-Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-JSON-Daten:"+ resulted_data},
#            {"role": "user", "content": "Im JSON-Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
#            }
#        ]
#    )
#    return response['choices'][0]['message']['content']

def generate_response(resulted_data):
    response = openai.ChatCompletion.create(
    model="gpt-4-0314",
    # engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
    temperature=0.8,
    messages=[
      {"role": "user",
       "content": "Du bist ein Analyst. Aus den bereitgestellten Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-List-Daten:" + resulted_data + "Im Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
      }
    ]
  )
    return response['choices'][0]['message']['content']


# Define the function to filter the accident data
def get_incident_data(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    #gpt filter
    filtered_incidents = [
        {
            'title': entry['title'],
            'description': entry['description']
        } 
        for entry in data['incidents'] 
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime
    ]
        
    resulted_data = json.dumps(filtered_incidents)
    
    print(resulted_data)
    
    #filtered_gpt_data = ''
    filtered_gpt_data = generate_response(resulted_data)
    
    return filtered_gpt_data



# Define the function to filter the line data 
def get_top10_lines(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    filtered_lines_incidents = []
    for entry in data['incidents']:
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime:
            filtered_lines_incidents.append(entry['lines'])

    from collections import Counter


    # Flatten the list of lists
    flattened_list = [item for sublist in filtered_lines_incidents for item in sublist]
    # Calculate the frequency of each element
    element_frequencies = Counter(flattened_list)
    # Get the top 10 most common elements and their frequencies
    top_10 = element_frequencies.most_common(10)

    return top_10

# Define the function to filter the accident time data
def get_accident_time_sum(data, start_time, end_time):
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)

    filtered_accident_sum = {"Verkehrsunfall": 0, "Falschparker": 0}
    #for entry in data['incidents']:
    for entry in data['incidents']:
        # Data validation for the 'start' and 'end' timestamps
        if 'start' not in entry or 'end' not in entry:
            #print("Missing 'start' or 'end' in entry:", entry)
            continue
        if not entry['start'] or not entry['end']:
            #print("Empty 'start' or 'end' timestamp in entry:", entry)
            continue

        try:
            start_incident_datetime = datetime.strptime(entry['start'], date_format)
            end_incident_datetime = datetime.strptime(entry['end'], date_format)
        except ValueError as e:
            #print(f"Error parsing timestamp in entry {entry}: {e}")
            continue

        # Check the incident title and compute the time sum
        if start_datetime <= start_incident_datetime <= end_datetime and ("Verkehrsunfall" in entry['title'] or "Falschparker" in entry['title']):
            if "Verkehrsunfall" in entry['title']:
                filtered_accident_sum['Verkehrsunfall'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60
            if "Falschparker" in entry['title']:
                filtered_accident_sum['Falschparker'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60

    filtered_accident_sum = {key: (round(value, 2) if isinstance(value, float) else value) for key, value in filtered_accident_sum.items()}

    return filtered_accident_sum


# Define the function to generate the date ranges
def get_date_ranges():
    # Initialize the end date to today's date at 00:00:00
    end_time = datetime.strptime('2018-08-16 00:00:00.00000','%Y-%m-%d %H:%M:%S.%f')
    # Define the time interval (1 week = 7 days)
    interval = timedelta(days=1)
    date_ranges = []
    # Generate the ranges
    for _ in range(3):
        start_time = end_time - interval
        date_ranges.append((start_time, end_time))
        end_time = start_time
    return date_ranges


# Define the function to generate the Subway_line data
def get_line_data(date_ranges):
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)

    #top 10 line data
    lines_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filetered_top_10_line_data = get_top10_lines(data, start_str, end_str)
        #print("Line data from data analysis:")
        lines_data_list.append(filetered_top_10_line_data)
    print(lines_data_list)
    return plot_line_data_to_html(lines_data_list)


# Define the function to generate the accident data
def get_accident_data(date_ranges):
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)

    #accident time sum daata
    accident_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filtered_accident_time_data = get_accident_time_sum(data, start_str, end_str)
        #print("Filtered Accident time data")
        accident_data_list.append(filtered_accident_time_data)
    print(accident_data_list)
    return plot_accident_data_to_html(accident_data_list)


# Define the function to generate the gpt data
def get_gpt_data(date_ranges):
    # Use the list of tuples in a for loop
    data = read_data()
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        
        print(f"Start: {start_str}, End: {end_str}")
        filtered_gpt_data = get_incident_data(data, start_str, end_str)
        
        print("Filtered GPT station data:")
        print(filtered_gpt_data)
        break
    return plot_gpt_data_to_html(filtered_gpt_data)

# Define the function to read the data
def read_data():
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)
    return data 
#get_line_data(get_date_ranges())
#get_accident_data(get_date_ranges())
#get_gpt_data(get_date_ranges())


# 2022-08-16 hypo-1 tem-0.8

In [24]:
get_gpt_data(get_date_ranges())

Start: 2018-08-15 00:00:00, End: 2018-08-16 00:00:00
[{"title": "U3 : Schadhaftes Fahrzeug", "description": "Wegen eines schadhaften Fahrzeuges werden die Z\u00c3\u0192\u00c2\u00bcge der Linie U3 zwischen den Stationen Zieglergasse und Volkstheater U \u00c3\u0192\u00c2\u00bcber Gleis 1 gef\u00c3\u0192\u00c2\u00bchrt.. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 23:45 Uhr!"}, {"title": "29A : Polizeieinsatz", "description": "Nach einer Fahrtbehinderung kommt es auf der Linie 29A zu unterschiedlichen Intervallen."}, {"title": "9 : Falschparker", "description": "Wegen eines Falschparkers im Bereich Hormayrgasse 18 ist die Linie 9 in Fahrtrichtung Wallri\u00c3\u0192\u00c2\u0178stra\u00c3\u0192\u00c2\u0178e an der Weiterfahrt gehindert. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 21:35 Uhr!"}, {"title": "13A :", "description": "Nach einer Fahrtbehinderung kommt es auf der Linie 13A zu unterschiedlichen Intervallen."}, {"title": "U1 : Polizeieinsatz", "des

In [25]:
import json
from dotenv import load_dotenv
from datetime import datetime, timedelta
import openai
import os
from pyecharts import *
#from plot import plot_line_data_to_html, plot_accident_data_to_html, plot_gpt_data_to_html 

load_dotenv()
# Configure the OpenAI API client
#openai.api_type = os.getenv("OPENAI_API_TYPE")
#openai.api_version = os.getenv("OPENAI_API_VERSION")
#openai.api_base = os.getenv("OPENAI_API_BASE")  # Your Azure OpenAI resource's endpoint value.
#openai.api_key = os.getenv("OPENAI_API_KEY")

# Define the function to generate the response
#def generate_response(resulted_data):
#    response = openai.ChatCompletion.create(
#        model="gpt-4",
        #engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
#        temperature = 0.4,
#        messages=[
#            {"role": "system", "content": "Du bist ein Analyse-Chatbot. Aus den bereitgestellten JSON-Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-JSON-Daten:"+ resulted_data},
#            {"role": "user", "content": "Im JSON-Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
#            }
#        ]
#    )
#    return response['choices'][0]['message']['content']

def generate_response(resulted_data):
    response = openai.ChatCompletion.create(
    model="gpt-4-0314",
    # engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
    temperature=0.9,
    messages=[
      {"role": "user",
       "content": "Du bist ein Analyst. Aus den bereitgestellten Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-List-Daten:" + resulted_data + "Im Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
      }
    ]
  )
    return response['choices'][0]['message']['content']


# Define the function to filter the accident data
def get_incident_data(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    #gpt filter
    filtered_incidents = [
        {
            'title': entry['title'],
            'description': entry['description']
        } 
        for entry in data['incidents'] 
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime
    ]
        
    resulted_data = json.dumps(filtered_incidents)
    
    print(resulted_data)
    
    #filtered_gpt_data = ''
    filtered_gpt_data = generate_response(resulted_data)
    
    return filtered_gpt_data



# Define the function to filter the line data 
def get_top10_lines(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    filtered_lines_incidents = []
    for entry in data['incidents']:
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime:
            filtered_lines_incidents.append(entry['lines'])

    from collections import Counter


    # Flatten the list of lists
    flattened_list = [item for sublist in filtered_lines_incidents for item in sublist]
    # Calculate the frequency of each element
    element_frequencies = Counter(flattened_list)
    # Get the top 10 most common elements and their frequencies
    top_10 = element_frequencies.most_common(10)

    return top_10

# Define the function to filter the accident time data
def get_accident_time_sum(data, start_time, end_time):
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)

    filtered_accident_sum = {"Verkehrsunfall": 0, "Falschparker": 0}
    #for entry in data['incidents']:
    for entry in data['incidents']:
        # Data validation for the 'start' and 'end' timestamps
        if 'start' not in entry or 'end' not in entry:
            #print("Missing 'start' or 'end' in entry:", entry)
            continue
        if not entry['start'] or not entry['end']:
            #print("Empty 'start' or 'end' timestamp in entry:", entry)
            continue

        try:
            start_incident_datetime = datetime.strptime(entry['start'], date_format)
            end_incident_datetime = datetime.strptime(entry['end'], date_format)
        except ValueError as e:
            #print(f"Error parsing timestamp in entry {entry}: {e}")
            continue

        # Check the incident title and compute the time sum
        if start_datetime <= start_incident_datetime <= end_datetime and ("Verkehrsunfall" in entry['title'] or "Falschparker" in entry['title']):
            if "Verkehrsunfall" in entry['title']:
                filtered_accident_sum['Verkehrsunfall'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60
            if "Falschparker" in entry['title']:
                filtered_accident_sum['Falschparker'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60

    filtered_accident_sum = {key: (round(value, 2) if isinstance(value, float) else value) for key, value in filtered_accident_sum.items()}

    return filtered_accident_sum


# Define the function to generate the date ranges
def get_date_ranges():
    # Initialize the end date to today's date at 00:00:00
    end_time = datetime.strptime('2018-08-16 00:00:00.00000','%Y-%m-%d %H:%M:%S.%f')
    # Define the time interval (1 week = 7 days)
    interval = timedelta(days=1)
    date_ranges = []
    # Generate the ranges
    for _ in range(3):
        start_time = end_time - interval
        date_ranges.append((start_time, end_time))
        end_time = start_time
    return date_ranges


# Define the function to generate the Subway_line data
def get_line_data(date_ranges):
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)

    #top 10 line data
    lines_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filetered_top_10_line_data = get_top10_lines(data, start_str, end_str)
        #print("Line data from data analysis:")
        lines_data_list.append(filetered_top_10_line_data)
    print(lines_data_list)
    return plot_line_data_to_html(lines_data_list)


# Define the function to generate the accident data
def get_accident_data(date_ranges):
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)

    #accident time sum daata
    accident_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filtered_accident_time_data = get_accident_time_sum(data, start_str, end_str)
        #print("Filtered Accident time data")
        accident_data_list.append(filtered_accident_time_data)
    print(accident_data_list)
    return plot_accident_data_to_html(accident_data_list)


# Define the function to generate the gpt data
def get_gpt_data(date_ranges):
    # Use the list of tuples in a for loop
    data = read_data()
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        
        print(f"Start: {start_str}, End: {end_str}")
        filtered_gpt_data = get_incident_data(data, start_str, end_str)
        
        print("Filtered GPT station data:")
        print(filtered_gpt_data)
        break
    return plot_gpt_data_to_html(filtered_gpt_data)

# Define the function to read the data
def read_data():
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)
    return data 
#get_line_data(get_date_ranges())
#get_accident_data(get_date_ranges())
#get_gpt_data(get_date_ranges())


# 2022-08-16 hypo-1 tem-0.9

In [26]:
get_gpt_data(get_date_ranges())

Start: 2018-08-15 00:00:00, End: 2018-08-16 00:00:00
[{"title": "U3 : Schadhaftes Fahrzeug", "description": "Wegen eines schadhaften Fahrzeuges werden die Z\u00c3\u0192\u00c2\u00bcge der Linie U3 zwischen den Stationen Zieglergasse und Volkstheater U \u00c3\u0192\u00c2\u00bcber Gleis 1 gef\u00c3\u0192\u00c2\u00bchrt.. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 23:45 Uhr!"}, {"title": "29A : Polizeieinsatz", "description": "Nach einer Fahrtbehinderung kommt es auf der Linie 29A zu unterschiedlichen Intervallen."}, {"title": "9 : Falschparker", "description": "Wegen eines Falschparkers im Bereich Hormayrgasse 18 ist die Linie 9 in Fahrtrichtung Wallri\u00c3\u0192\u00c2\u0178stra\u00c3\u0192\u00c2\u0178e an der Weiterfahrt gehindert. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 21:35 Uhr!"}, {"title": "13A :", "description": "Nach einer Fahrtbehinderung kommt es auf der Linie 13A zu unterschiedlichen Intervallen."}, {"title": "U1 : Polizeieinsatz", "des

In [27]:
import json
from dotenv import load_dotenv
from datetime import datetime, timedelta
import openai
import os
from pyecharts import *
#from plot import plot_line_data_to_html, plot_accident_data_to_html, plot_gpt_data_to_html 

load_dotenv()
# Configure the OpenAI API client
#openai.api_type = os.getenv("OPENAI_API_TYPE")
#openai.api_version = os.getenv("OPENAI_API_VERSION")
#openai.api_base = os.getenv("OPENAI_API_BASE")  # Your Azure OpenAI resource's endpoint value.
#openai.api_key = os.getenv("OPENAI_API_KEY")

# Define the function to generate the response
#def generate_response(resulted_data):
#    response = openai.ChatCompletion.create(
#        model="gpt-4",
        #engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
#        temperature = 0.4,
#        messages=[
#            {"role": "system", "content": "Du bist ein Analyse-Chatbot. Aus den bereitgestellten JSON-Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-JSON-Daten:"+ resulted_data},
#            {"role": "user", "content": "Im JSON-Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
#            }
#        ]
#    )
#    return response['choices'][0]['message']['content']

def generate_response(resulted_data):
    response = openai.ChatCompletion.create(
    model="gpt-4-0314",
    # engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
    temperature=1.0,
    messages=[
      {"role": "user",
       "content": "Du bist ein Analyst. Aus den bereitgestellten Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-List-Daten:" + resulted_data + "Im Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
      }
    ]
  )
    return response['choices'][0]['message']['content']


# Define the function to filter the accident data
def get_incident_data(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    #gpt filter
    filtered_incidents = [
        {
            'title': entry['title'],
            'description': entry['description']
        } 
        for entry in data['incidents'] 
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime
    ]
        
    resulted_data = json.dumps(filtered_incidents)
    
    print(resulted_data)
    
    #filtered_gpt_data = ''
    filtered_gpt_data = generate_response(resulted_data)
    
    return filtered_gpt_data



# Define the function to filter the line data 
def get_top10_lines(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    filtered_lines_incidents = []
    for entry in data['incidents']:
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime:
            filtered_lines_incidents.append(entry['lines'])

    from collections import Counter


    # Flatten the list of lists
    flattened_list = [item for sublist in filtered_lines_incidents for item in sublist]
    # Calculate the frequency of each element
    element_frequencies = Counter(flattened_list)
    # Get the top 10 most common elements and their frequencies
    top_10 = element_frequencies.most_common(10)

    return top_10

# Define the function to filter the accident time data
def get_accident_time_sum(data, start_time, end_time):
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)

    filtered_accident_sum = {"Verkehrsunfall": 0, "Falschparker": 0}
    #for entry in data['incidents']:
    for entry in data['incidents']:
        # Data validation for the 'start' and 'end' timestamps
        if 'start' not in entry or 'end' not in entry:
            #print("Missing 'start' or 'end' in entry:", entry)
            continue
        if not entry['start'] or not entry['end']:
            #print("Empty 'start' or 'end' timestamp in entry:", entry)
            continue

        try:
            start_incident_datetime = datetime.strptime(entry['start'], date_format)
            end_incident_datetime = datetime.strptime(entry['end'], date_format)
        except ValueError as e:
            #print(f"Error parsing timestamp in entry {entry}: {e}")
            continue

        # Check the incident title and compute the time sum
        if start_datetime <= start_incident_datetime <= end_datetime and ("Verkehrsunfall" in entry['title'] or "Falschparker" in entry['title']):
            if "Verkehrsunfall" in entry['title']:
                filtered_accident_sum['Verkehrsunfall'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60
            if "Falschparker" in entry['title']:
                filtered_accident_sum['Falschparker'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60

    filtered_accident_sum = {key: (round(value, 2) if isinstance(value, float) else value) for key, value in filtered_accident_sum.items()}

    return filtered_accident_sum


# Define the function to generate the date ranges
def get_date_ranges():
    # Initialize the end date to today's date at 00:00:00
    end_time = datetime.strptime('2018-08-16 00:00:00.00000','%Y-%m-%d %H:%M:%S.%f')
    # Define the time interval (1 week = 7 days)
    interval = timedelta(days=1)
    date_ranges = []
    # Generate the ranges
    for _ in range(3):
        start_time = end_time - interval
        date_ranges.append((start_time, end_time))
        end_time = start_time
    return date_ranges


# Define the function to generate the Subway_line data
def get_line_data(date_ranges):
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)

    #top 10 line data
    lines_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filetered_top_10_line_data = get_top10_lines(data, start_str, end_str)
        #print("Line data from data analysis:")
        lines_data_list.append(filetered_top_10_line_data)
    print(lines_data_list)
    return plot_line_data_to_html(lines_data_list)


# Define the function to generate the accident data
def get_accident_data(date_ranges):
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)

    #accident time sum daata
    accident_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filtered_accident_time_data = get_accident_time_sum(data, start_str, end_str)
        #print("Filtered Accident time data")
        accident_data_list.append(filtered_accident_time_data)
    print(accident_data_list)
    return plot_accident_data_to_html(accident_data_list)


# Define the function to generate the gpt data
def get_gpt_data(date_ranges):
    # Use the list of tuples in a for loop
    data = read_data()
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        
        print(f"Start: {start_str}, End: {end_str}")
        filtered_gpt_data = get_incident_data(data, start_str, end_str)
        
        print("Filtered GPT station data:")
        print(filtered_gpt_data)
        break
    return plot_gpt_data_to_html(filtered_gpt_data)

# Define the function to read the data
def read_data():
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)
    return data 
#get_line_data(get_date_ranges())
#get_accident_data(get_date_ranges())
#get_gpt_data(get_date_ranges())


# 2022-08-16 hypo-1 tem-1.0

In [28]:
get_gpt_data(get_date_ranges())

Start: 2018-08-15 00:00:00, End: 2018-08-16 00:00:00
[{"title": "U3 : Schadhaftes Fahrzeug", "description": "Wegen eines schadhaften Fahrzeuges werden die Z\u00c3\u0192\u00c2\u00bcge der Linie U3 zwischen den Stationen Zieglergasse und Volkstheater U \u00c3\u0192\u00c2\u00bcber Gleis 1 gef\u00c3\u0192\u00c2\u00bchrt.. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 23:45 Uhr!"}, {"title": "29A : Polizeieinsatz", "description": "Nach einer Fahrtbehinderung kommt es auf der Linie 29A zu unterschiedlichen Intervallen."}, {"title": "9 : Falschparker", "description": "Wegen eines Falschparkers im Bereich Hormayrgasse 18 ist die Linie 9 in Fahrtrichtung Wallri\u00c3\u0192\u00c2\u0178stra\u00c3\u0192\u00c2\u0178e an der Weiterfahrt gehindert. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 21:35 Uhr!"}, {"title": "13A :", "description": "Nach einer Fahrtbehinderung kommt es auf der Linie 13A zu unterschiedlichen Intervallen."}, {"title": "U1 : Polizeieinsatz", "des

In [29]:
import json
from dotenv import load_dotenv
from datetime import datetime, timedelta
import openai
import os
from pyecharts import *
#from plot import plot_line_data_to_html, plot_accident_data_to_html, plot_gpt_data_to_html 

load_dotenv()
# Configure the OpenAI API client
#openai.api_type = os.getenv("OPENAI_API_TYPE")
#openai.api_version = os.getenv("OPENAI_API_VERSION")
#openai.api_base = os.getenv("OPENAI_API_BASE")  # Your Azure OpenAI resource's endpoint value.
#openai.api_key = os.getenv("OPENAI_API_KEY")

# Define the function to generate the response
#def generate_response(resulted_data):
#    response = openai.ChatCompletion.create(
#        model="gpt-4",
        #engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
#        temperature = 0.4,
#        messages=[
#            {"role": "system", "content": "Du bist ein Analyse-Chatbot. Aus den bereitgestellten JSON-Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-JSON-Daten:"+ resulted_data},
#            {"role": "user", "content": "Im JSON-Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
#            }
#        ]
#    )
#    return response['choices'][0]['message']['content']

def generate_response(resulted_data):
    response = openai.ChatCompletion.create(
    model="gpt-4-0314",
    # engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
    temperature=0,
    messages=[
      {"role": "user",
       "content": "Du bist ein Analyst. Aus den bereitgestellten Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-List-Daten:" + resulted_data + "Im Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
      }
    ]
  )
    return response['choices'][0]['message']['content']


# Define the function to filter the accident data
def get_incident_data(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    #gpt filter
    filtered_incidents = [
        {
            'title': entry['title'],
            'description': entry['description']
        } 
        for entry in data['incidents'] 
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime
    ]
        
    resulted_data = json.dumps(filtered_incidents)
    
    print(resulted_data)
    
    #filtered_gpt_data = ''
    filtered_gpt_data = generate_response(resulted_data)
    
    return filtered_gpt_data



# Define the function to filter the line data 
def get_top10_lines(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    filtered_lines_incidents = []
    for entry in data['incidents']:
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime:
            filtered_lines_incidents.append(entry['lines'])

    from collections import Counter


    # Flatten the list of lists
    flattened_list = [item for sublist in filtered_lines_incidents for item in sublist]
    # Calculate the frequency of each element
    element_frequencies = Counter(flattened_list)
    # Get the top 10 most common elements and their frequencies
    top_10 = element_frequencies.most_common(10)

    return top_10

# Define the function to filter the accident time data
def get_accident_time_sum(data, start_time, end_time):
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)

    filtered_accident_sum = {"Verkehrsunfall": 0, "Falschparker": 0}
    #for entry in data['incidents']:
    for entry in data['incidents']:
        # Data validation for the 'start' and 'end' timestamps
        if 'start' not in entry or 'end' not in entry:
            #print("Missing 'start' or 'end' in entry:", entry)
            continue
        if not entry['start'] or not entry['end']:
            #print("Empty 'start' or 'end' timestamp in entry:", entry)
            continue

        try:
            start_incident_datetime = datetime.strptime(entry['start'], date_format)
            end_incident_datetime = datetime.strptime(entry['end'], date_format)
        except ValueError as e:
            #print(f"Error parsing timestamp in entry {entry}: {e}")
            continue

        # Check the incident title and compute the time sum
        if start_datetime <= start_incident_datetime <= end_datetime and ("Verkehrsunfall" in entry['title'] or "Falschparker" in entry['title']):
            if "Verkehrsunfall" in entry['title']:
                filtered_accident_sum['Verkehrsunfall'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60
            if "Falschparker" in entry['title']:
                filtered_accident_sum['Falschparker'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60

    filtered_accident_sum = {key: (round(value, 2) if isinstance(value, float) else value) for key, value in filtered_accident_sum.items()}

    return filtered_accident_sum


# Define the function to generate the date ranges
def get_date_ranges():
    # Initialize the end date to today's date at 00:00:00
    end_time = datetime.strptime('2018-08-16 00:00:00.00000','%Y-%m-%d %H:%M:%S.%f')
    # Define the time interval (1 week = 7 days)
    interval = timedelta(days=1)
    date_ranges = []
    # Generate the ranges
    for _ in range(3):
        start_time = end_time - interval
        date_ranges.append((start_time, end_time))
        end_time = start_time
    return date_ranges


# Define the function to generate the Subway_line data
def get_line_data(date_ranges):
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)

    #top 10 line data
    lines_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filetered_top_10_line_data = get_top10_lines(data, start_str, end_str)
        #print("Line data from data analysis:")
        lines_data_list.append(filetered_top_10_line_data)
    print(lines_data_list)
    return plot_line_data_to_html(lines_data_list)


# Define the function to generate the accident data
def get_accident_data(date_ranges):
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)

    #accident time sum daata
    accident_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filtered_accident_time_data = get_accident_time_sum(data, start_str, end_str)
        #print("Filtered Accident time data")
        accident_data_list.append(filtered_accident_time_data)
    print(accident_data_list)
    return plot_accident_data_to_html(accident_data_list)


# Define the function to generate the gpt data
def get_gpt_data(date_ranges):
    # Use the list of tuples in a for loop
    data = read_data()
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        
        print(f"Start: {start_str}, End: {end_str}")
        filtered_gpt_data = get_incident_data(data, start_str, end_str)
        
        print("Filtered GPT station data:")
        print(filtered_gpt_data)
        break
    return plot_gpt_data_to_html(filtered_gpt_data)

# Define the function to read the data
def read_data():
    with open('Wiener_2013-2023.json', 'r') as file:
        data = json.load(file)
    return data 
#get_line_data(get_date_ranges())
#get_accident_data(get_date_ranges())
#get_gpt_data(get_date_ranges())


# 2022-08-16 hypo-1 tem-0.0

In [30]:
get_gpt_data(get_date_ranges())

Start: 2018-08-15 00:00:00, End: 2018-08-16 00:00:00
[{"title": "U3 : Schadhaftes Fahrzeug", "description": "Wegen eines schadhaften Fahrzeuges werden die Z\u00c3\u0192\u00c2\u00bcge der Linie U3 zwischen den Stationen Zieglergasse und Volkstheater U \u00c3\u0192\u00c2\u00bcber Gleis 1 gef\u00c3\u0192\u00c2\u00bchrt.. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 23:45 Uhr!"}, {"title": "29A : Polizeieinsatz", "description": "Nach einer Fahrtbehinderung kommt es auf der Linie 29A zu unterschiedlichen Intervallen."}, {"title": "9 : Falschparker", "description": "Wegen eines Falschparkers im Bereich Hormayrgasse 18 ist die Linie 9 in Fahrtrichtung Wallri\u00c3\u0192\u00c2\u0178stra\u00c3\u0192\u00c2\u0178e an der Weiterfahrt gehindert. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 21:35 Uhr!"}, {"title": "13A :", "description": "Nach einer Fahrtbehinderung kommt es auf der Linie 13A zu unterschiedlichen Intervallen."}, {"title": "U1 : Polizeieinsatz", "des